# Лаб. работа 2


Домашнее задание Яндекса.

In [15]:
import warnings
warnings.filterwarnings('ignore')
import gensim
import numpy as np
from gensim.models import KeyedVectors

Подгружаем эмбединги для русского и украинского языков. Поскольку компьютер не тянет полный объем, прогружаем частями(через параметр лимин)

In [16]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec", limit = 50000)
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec", limit = 50000)

Теперь посмотрим "ближайших соседей" в рамках языков. Изучим схожесть языков.

In [17]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0000001192092896),
 ('июль', 0.938315212726593),
 ('сентябрь', 0.9240026473999023),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095540046691895),
 ('ноябрь', 0.893003523349762),
 ('апрель', 0.8729085326194763),
 ('декабрь', 0.86525559425354),
 ('март', 0.8545794486999512),
 ('февраль', 0.8401417136192322)]

In [18]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 1.0000003576278687),
 ('липень', 0.9096442461013794),
 ('вересень', 0.9016972780227661),
 ('червень', 0.8992522358894348),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787632584571838),
 ('квітень', 0.8592804074287415),
 ('грудень', 0.8586865067481995),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256436586380005)]

In [19]:
ru_emb.most_similar([uk_emb["серпень"]])

[('подход', 0.22305867075920105),
 ('подхода', 0.21423815190792084),
 ('аспектах', 0.21134454011917114),
 ('двустороннего', 0.2105553299188614),
 ('прикладного', 0.2064727395772934),
 ('Правовое', 0.2055036574602127),
 ('Профилактика', 0.20540721714496613),
 ('сторона', 0.2051437348127365),
 ('стиль', 0.2022884339094162),
 ('аспекте', 0.20203247666358948)]

Прогружаем словари для тренировочных пар слов для отработки в ролитестовой и тренировочной выборок

In [20]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [21]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

# Векторное отображение

Пусть $x_i \in R^d - $ представление слова $i$ в исходном языке, и $y_i \in R^d - $ представление его перевода. Наша задача $-$ отыскать линейное преобразование $W$, минимизирующее евклидово расстояние между векторами $W x_i$ и $y_i$ для некоторого подмножества векторных представлений слов. Известно, что искомое преобразование является линейной регрессией.

In [22]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression(fit_intercept=False).fit(X_train, Y_train)

Рассмотрим соседей русского "августа" на украинском(звучит как серпень) после линейного преобразования

In [23]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8566749691963196),
 ('март', 0.8490580320358276),
 ('февраль', 0.8444869518280029),
 ('июнь', 0.843612015247345),
 ('сентябрь', 0.8414735794067383),
 ('ноябрь', 0.8390121459960938),
 ('октябрь', 0.83610600233078),
 ('июль', 0.8214761018753052),
 ('август', 0.8157680034637451),
 ('декабрь', 0.8157303333282471)]

Верный ответ есть в списке но находится на 9м месте. В роли меры качества меры будем использовать топ-1 топ-5, топ-10(для каждого преобразованного украинского набора считаем, сколько правильный целевых пар найдено в вершине N относительно самых близских соседей в русском наборе). Посчитаем количество таких правильных переводов.

In [24]:
def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        most_similar = ru_emb.most_similar([mapped_vectors[i]])[:topn]
        num_matches += int(any([ru == word for word, _ in most_similar]))
    precision_val = num_matches / len(pairs)
    return precision_val

In [25]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [26]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [28]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [29]:
print(precision_top1)
print(precision_top5)

0.6677966101694915
0.8372881355932204


Если загрузиь больше данные, значение тоже должно возрасти.

# Оптимизация

Теперь потребуем, чтобы наше линейное преобразование было ортогональным.

In [30]:
from sklearn.decomposition import TruncatedSVD
def learn_transform(X_train, Y_train):
    U, S, Vh = np.linalg.svd(np.dot(X_train.T, Y_train))
    return np.dot(U, Vh)

In [31]:
W = learn_transform(X_train, Y_train)

print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6779661016949152
0.8440677966101695


Результаты немного улучшились

# Перевод с украинского на русский

Теперь попробуем сделать простой переводсик, который для каждого слова исходного языка будет отыскивать ближайшее слово в векторном пространстве целевого языка.

In [32]:
def translate(sentence):
    words = sentence.split(" ")
    translation = []
    for word in words:
        try:
            emb = uk_emb[word]
            translation.append(ru_emb.most_similar([np.matmul(emb, W)], topn=1)[0][0])           
        except:
            translation.append(word)
    return " ".join(translation)

In [33]:
print(translate("Скоро Новий Рік"))

Скоро Новый Год


In [34]:
print(translate("Сподіваюся, що перекладач працює правильно"))

Сподіваюся, что переводчик работает правильно


In [35]:
print(translate("Ну, хоча б приблизно правильно"))

Ну, хотя бы примерно правильно


Мы получили относительное качество перевода. Причиной этому может быть ограниченный объем векторных представлений. Теперь посмотрим работу на уровне текста.

In [36]:
with open("fairy_tale.txt", "r", encoding='utf8') as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [37]:
uk_sentences

['лисичка - сестричка і вовк - панібрат',
 'як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !',
 'дай мені огня " .',
 'а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .',
 'прибігла на поле , аж там пасуть хлопці бичків .',
 'вона і каже їм : " ей , хлопці !',
 'проміняйте мені бичка - третячка за маковий пирожок " .',
 'тії согласились ; так вона їм говорить : " смотріть же , ви не їжте зараз сього пирожка , а тоді уже розломите , як я заведу бичка за могилку ; а то ви його ні за що не розломите " .',
 'бачите вже - лисичка таки собі була розумна , що хоть кого да обманить .',
 'тії хлопці так і зробили , а лисичка як зайшла за могилу , да зараз у ліс і повернула , 

In [39]:
def translate(sentence, src_emb, dst_emb):
    words = sentence.split(' ')
    translation = []
    for word in words:
        try:
            ru_word = dst_emb.most_similar([np.matmul(src_emb[word], W)], topn=1)[0][0]
            translation.append(ru_word)           
        except:
            translation.append(word)
    return ' '.join(translation)

def uk_ru_translate(sentence):
    return translate(sentence, uk_emb, ru_emb)

In [40]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, uk_ru_translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – панібрат

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка , че и пошла раз к однії бабы добувать огню ; вошла во дом че и говорит : " хороший день тебе , бабусю !

src: дай мені огня " .
dst: дай мне огня " .

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а баба только что вийняла со печи пирожок со маком , сладкий , че и положила , чтобы он прохолов ; а лисичка ж и підгледала , че тілько что баба нахилилась во печь , чтобы достать огня , то лисичка сейчас ухватила пирожок че и драла со дома , че , біжучи , весь мак со его виїла , а туда мусор н

src: а собі на думці : " подожди , і я зроблю з тобою таку штуку , як і ти зо мною " .
dst: а себе по мнении : " подожди , и мной сделаю со тобой такую штуку , как и ты За мной " .

src: - " як же ти ловила ? "
dst: – " как то ты ловила ? "

src: - " так , вовчику , уложила хвостик в ополонку , вожу тихенько да й кажу ; ловися , рибка , мала і велика !
dst: – " так , вовчику , уложила хвостик во ополонку , вожу тихенько че и говорю ; ловися , рибка , имела и большая !

src: коли хочеш , то і ти піди , налови собі " .
dst: когда хочу , то и ты піди , налови себе " .

src: він побіг да зробив так , як казала лисичка .
dst: он побіг че сделал так , как говорила лисичка .

src: а лисичка стала за деревом да й дивиться ; коли у вовчика зовсім хвостик примерз , вона тоді побігла в село да й кричить : " ідіть , люди , вбивайте вовка ! "
dst: а лисичка стала за деревом че и смотрит ; когда во вовчика совсем хвостик примерз , она тогда побігла во село че и кричит : " ідіть , люди , вбивайте вол

Переводчик работает. Но результаты далеки от совершенства.

# Spanish / English

Теперь нужно сделать переводчик для новой пары языков. Сначала загружаем векторные представления слов испанского и английского языков.

In [41]:
en_emb = KeyedVectors.load_word2vec_format("cc.en.300.vec",binary=False, limit = 10000)
es_emb = KeyedVectors.load_word2vec_format("cc.es.300.vec",binary=False, limit = 10000)

Перевод с испанского на английскийю
Сначала загрузим пары слов из текстовых файлов

In [50]:
def load_word_pairs(filename):
    es_en_pairs = []
    es_vectors = []
    en_vectors = []
    
    with open(filename, "r") as inpf:
        for line in inpf:
            es, en = line.rstrip().split(" ")
            if es not in es_emb or en not in en_emb:
                continue
            es_en_pairs.append((es, en))
            es_vectors.append(es_emb[es])
            en_vectors.append(en_emb[en])
            
    return es_en_pairs, np.array(es_vectors), np.array(en_vectors)

In [51]:
es_en_train, X_train, Y_train = load_word_pairs("es-en.0-5000.txt")

In [52]:
es_en_test, X_test, Y_test = load_word_pairs("es-en.5000-6500.txt")

In [53]:
def learn_transform(X, Y):
    U, s, V = np.linalg.svd(np.matmul(X_train.T,Y_train))
    W = np.matmul(U,V)
    return W

Строим матрицы отображений между пространствами векторных репрезентаций.

In [54]:
M1 = learn_transform(X_train[:300], Y_train[:300])
M2 = learn_transform(X_train[300:601], Y_train[300:601])

In [58]:
def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        mapped_vector = mapped_vectors[i]
        if ru in [x for (x,_) in en_emb.most_similar([mapped_vector],topn=topn)]:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [59]:
print("pair1 M1: {}".format(precision(es_en_train[:300], np.matmul(X_train[:300], M1))))
print("pair1 M2: {}".format(precision(es_en_train[:300], np.matmul(X_train[:300], M2))))
print("pair2 M1: {}".format(precision(es_en_train[300:601], np.matmul(X_train[300:601], M1))))
print("pair2 M2: {}".format(precision(es_en_train[300:601], np.matmul(X_train[300:601], M2))))

pair1 M1: 0.5266666666666666
pair1 M2: 0.5266666666666666
pair2 M1: 0.5481727574750831
pair2 M2: 0.5481727574750831


Для пары es - en языки получили точность 55%

Вычислим норму фробениуса для матрицы $|M1-M2|$:

In [60]:
M1_M2 = np.linalg.norm((M1-M2), ord ='fro')
M1_M2

0.0

Вычислим отношение фробениусовых норм $\frac{|M1-M2|}{|M1|}$:

In [61]:
M1_M2/np.linalg.norm(M1, ord ='fro')

0.0

Теперь проверим каждую из матриц $M_1$ и $M_2$ на ортогональность. Для этого вычислим произведения $M_1 \cdot M_1 ^ T$ и $M_2 \cdot M_2 ^ T$.

In [62]:
print(np.matmul(M1,M1.T))

[[ 9.9999994e-01  3.4284312e-08  1.0011718e-08 ...  0.0000000e+00
   7.5669959e-10  0.0000000e+00]
 [ 3.4284312e-08  1.0000004e+00 -1.8335413e-09 ...  7.0431270e-09
   1.1234079e-08  1.9092113e-08]
 [ 1.0011718e-08 -1.8335413e-09  1.0000000e+00 ...  6.5192580e-09
  -2.0489097e-08  3.7252903e-09]
 ...
 [ 0.0000000e+00  7.0431270e-09  6.5192580e-09 ...  9.9999958e-01
  -2.5611371e-09 -1.8626451e-08]
 [ 7.5669959e-10  1.1234079e-08 -2.0489097e-08 ... -2.5611371e-09
   1.0000000e+00 -1.8626451e-08]
 [ 0.0000000e+00  1.9092113e-08  3.7252903e-09 ... -1.8626451e-08
  -1.8626451e-08  9.9999994e-01]]


In [63]:
print(np.matmul(M2,M2.T))

[[ 9.9999994e-01  3.4284312e-08  1.0011718e-08 ...  0.0000000e+00
   7.5669959e-10  0.0000000e+00]
 [ 3.4284312e-08  1.0000004e+00 -1.8335413e-09 ...  7.0431270e-09
   1.1234079e-08  1.9092113e-08]
 [ 1.0011718e-08 -1.8335413e-09  1.0000000e+00 ...  6.5192580e-09
  -2.0489097e-08  3.7252903e-09]
 ...
 [ 0.0000000e+00  7.0431270e-09  6.5192580e-09 ...  9.9999958e-01
  -2.5611371e-09 -1.8626451e-08]
 [ 7.5669959e-10  1.1234079e-08 -2.0489097e-08 ... -2.5611371e-09
   1.0000000e+00 -1.8626451e-08]
 [ 0.0000000e+00  1.9092113e-08  3.7252903e-09 ... -1.8626451e-08
  -1.8626451e-08  9.9999994e-01]]


Видим, что все элементы, кроме диагональных, очень близки к нулю, а диагональные элементы очень близки к единице.

Значит, можно считать, что $M_1 \cdot M_1 ^ T = E$ и $M_2 \cdot M_2 ^ T = E$ и каждая из матриц $M_1, M_2$ ортогональна.

# Перевод с английского на испанский

Алгоритм аналогичен.


In [79]:
def load_word_pairs(filename):
    en_es_pairs = []
    en_vectors = []
    es_vectors = []
    
    with open(filename, "r") as inpf:
        for line in inpf:
            en, es = line.rstrip().split(" ")
            if en not in en_emb or es not in es_emb:
                continue
            en_es_pairs.append((en, es))
            en_vectors.append(en_emb[en])
            es_vectors.append(es_emb[es])
            
    return en_es_pairs, np.array(en_vectors), np.array(es_vectors)

In [80]:
en_es_train, X_train, Y_train = load_word_pairs("en-es.0-5000.txt")


In [81]:
en_es_test, X_test, Y_test = load_word_pairs("en-es.5000-6500.txt")

Получим две матрицы преобразования M1 и M2 для первых 300 пар и следующих 300 пар соответственно:

In [85]:
M1 = learn_transform(X_train[:300], Y_train[:300])
M2 = learn_transform(X_train[300:601], Y_train[300:601])

In [86]:
def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        mapped_vector = mapped_vectors[i]
        if ru in [x for (x, _) in es_emb.most_similar([mapped_vector], topn=topn)]:
            num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [87]:
print("pair1 M1: {}".format(precision(en_es_train[:300], np.matmul(X_train[:300], M1))))
print("pair1 M2: {}".format(precision(en_es_train[:300], np.matmul(X_train[:300], M2))))
print("pair2 M1: {}".format(precision(en_es_train[300:601], np.matmul(X_train[300:601], M1))))
print("pair2 M2: {}".format(precision(en_es_train[300:601], np.matmul(X_train[300:601], M2))))

pair1 M1: 0.3566666666666667
pair1 M2: 0.3566666666666667
pair2 M1: 0.3521594684385382
pair2 M2: 0.3521594684385382


Для пары en - es языки получили точность 36%

Вычислим норму фробениуса для матрицы  $|M1-M2|$:

In [89]:
M1_M2 = np.linalg.norm((M1-M2), ord = "fro")
M1_M2

0.0

Вычислим отношение фробениусовых норм |M1-M2|/|M1|

In [90]:
M1_M2/np.linalg.norm(M1, ord = "fro")

0.0

Проверим ортогональность матриц:

In [91]:
print(np.matmul(M1, M1.T))

[[ 1.0000008e+00 -6.7811925e-09  1.3693352e-08 ... -4.2273314e-09
   3.7023710e-08 -2.0023435e-08]
 [-6.7811925e-09  1.0000010e+00 -5.5297278e-09 ... -2.0489097e-08
   8.7893568e-09 -3.7252903e-09]
 [ 1.3693352e-08 -5.5297278e-09  1.0000004e+00 ...  1.9965228e-08
   1.2747478e-08 -7.4505806e-09]
 ...
 [-4.2273314e-09 -2.0489097e-08  1.9965228e-08 ...  1.0000001e+00
  -1.4610123e-08  2.2351742e-08]
 [ 3.7023710e-08  8.7893568e-09  1.2747478e-08 ... -1.4610123e-08
   1.0000001e+00  6.5192580e-09]
 [-2.0023435e-08 -3.7252903e-09 -7.4505806e-09 ...  2.2351742e-08
   6.5192580e-09  1.0000001e+00]]


In [92]:
print(np.matmul(M2, M2.T))

[[ 1.0000008e+00 -6.7811925e-09  1.3693352e-08 ... -4.2273314e-09
   3.7023710e-08 -2.0023435e-08]
 [-6.7811925e-09  1.0000010e+00 -5.5297278e-09 ... -2.0489097e-08
   8.7893568e-09 -3.7252903e-09]
 [ 1.3693352e-08 -5.5297278e-09  1.0000004e+00 ...  1.9965228e-08
   1.2747478e-08 -7.4505806e-09]
 ...
 [-4.2273314e-09 -2.0489097e-08  1.9965228e-08 ...  1.0000001e+00
  -1.4610123e-08  2.2351742e-08]
 [ 3.7023710e-08  8.7893568e-09  1.2747478e-08 ... -1.4610123e-08
   1.0000001e+00  6.5192580e-09]
 [-2.0023435e-08 -3.7252903e-09 -7.4505806e-09 ...  2.2351742e-08
   6.5192580e-09  1.0000001e+00]]
